# Analyse des accidents sur voie publique en FRANCE

Les données correspondes aux accidents corporels sur la circulation publique de France en 2023. Les informations ont été recueillie par les unités de forces publiques qui ont intervenues. 

Les données sont disponibles sous 4 fichiers : 
- caractérstiques : informations sur les caractéristiques de l'accidents  
- lieux : informations sur le lieux de l'accidents
- véhicules : informations sur les véhicules impliqués
- usagers : informations sur les personnes impliqués

Chaque accidents est identifier par un numéro présent dans chaque fichier.

### Objecitf : comprendre les accidents pour pouvoir cibler les futures préventions routière, travaux etc...

- Classifier les accidents sous différentes catégories.
- Déterminer les variables explicatives les plus corrélés aux accidents mortels.
- Plan d'action pour certaines départements (dep = var expliqué)

## Analyse exploratrice des données 

### Importation

In [280]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [281]:
caract = pd.read_csv("acc_routes/caract-2023.csv", sep=";")
lieux = pd.read_csv("acc_routes/lieux-2023.csv",sep=";")
usagers = pd.read_csv("acc_routes/usagers-2023.csv",sep=";")
vehicules = pd.read_csv("acc_routes/vehicules-2023.csv",sep=";")

C:\Users\benfo\AppData\Local\Temp\ipykernel_11352\2705059668.py:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  lieux = pd.read_csv("acc_routes/lieux-2023.csv",sep=";")


On vérifie la taille des ensembles

In [282]:
print(caract.shape)
print(lieux.shape)
print(usagers.shape)
print(vehicules.shape)

(54822, 15)
(70860, 18)
(125789, 16)
(93585, 11)


-> On remarque que les ensembles ont des tailles différentes. On explique cela par la dublication d'enregistrements si l'accident concerne plusieurs lieux,véhicules et personnes.

----

### Pré-traitement 

On va s'intérresser à chaque variables qui peuvent être utiles dans notre analyse.


#### CARACTERISTIQUE : 

On souhaite garder : jour, mois, an, hrmn, lum, agg, int, atm, col, dep

In [283]:
caract = caract.drop(columns=["adr",'lat','long','com'])

Valeurs dupliqués 

In [284]:
caract.duplicated().value_counts()

False    54822
Name: count, dtype: int64

Il y en a aucune

Types des données

In [285]:
caract.dtypes

Num_Acc     int64
jour        int64
mois        int64
an          int64
hrmn       object
lum         int64
dep        object
agg         int64
int         int64
atm         int64
col         int64
dtype: object

Il y a deux types object

On va regrouper les quatres variables : hrmn, jour, mois et an en 1 datetime. Cela va nous permettre de facilement accéder à toutes les infos

In [286]:
caract["datetime"] = pd.to_datetime(
    caract['an'].astype(str) + '-' +
    caract['mois'].astype(str) + '-' +
    caract['jour'].astype(str) + ' ' +
    caract['hrmn'])

In [287]:
caract["datetime"][0].hour

6

In [288]:
caract = caract.drop(columns=["hrmn","an","mois",'jour'])

On va convertir les départements avec caractères en entier

In [289]:
valeurs_problemes = caract.loc[~caract["dep"].str.isnumeric(), "dep"].unique()
valeurs_problemes

array(['2B', '2A'], dtype=object)

Les départements de corse possèdent des caractères, on va les traîter spécifiquement

In [290]:
caract["dep"] = caract[ "dep"].replace({"2B": "222", "2A": "221"}).apply(pd.to_numeric, errors='coerce')

Vérification

In [291]:
caract.dtypes

Num_Acc              int64
lum                  int64
dep                  int64
agg                  int64
int                  int64
atm                  int64
col                  int64
datetime    datetime64[ns]
dtype: object

Valeurs manquantes

In [292]:
print(caract.isna().sum())

Num_Acc     0
lum         0
dep         0
agg         0
int         0
atm         0
col         0
datetime    0
dtype: int64


Autres valeurs manquantes, indiqué par -1 (non renseigné)

In [293]:
print((caract["col"]== -1).sum())
print((caract["atm"]== -1).sum())

13
4


In [294]:
def remplacer_mediane(df,cols):
    for col in cols:
        m_value = df[col].median()
        df[col] = df[col].fillna(int(m_value))
        df[col] = df[col].replace({-1:int(m_value)})
    return df

#### LIEUX :

On souhaite garder : catr, circ, nbv, prof, plan, lartpc, larrrout, surf, infra, situ ,vma

In [296]:
lieux = lieux.drop(columns=["voie",'v1','v2','vosp','pr','pr1'])

Types

In [297]:
lieux.dtypes

Num_Acc     int64
catr        int64
circ        int64
nbv        object
prof        int64
plan        int64
lartpc     object
larrout    object
surf        int64
infra       int64
situ        int64
vma         int64
dtype: object

Il y a trois types en object

In [298]:
valeurs_problemes = lieux.loc[~lieux["nbv"].str.isnumeric(), "nbv"].unique()
valeurs_problemes

array([' -1', '#VALEURMULTI'], dtype=object)

In [299]:
lieux[["lartpc", "larrout", "nbv"]] = lieux[["lartpc", "larrout", "nbv"]].apply(lambda x: x.str.replace(',', '.'))
lieux[["lartpc", "larrout", "nbv"]] = lieux[["lartpc", "larrout", "nbv"]].apply(pd.to_numeric, errors='coerce')


Valeurs manquantes

In [300]:
print(lieux.isna().sum())

Num_Acc        0
catr           0
circ           0
nbv           54
prof           0
plan           0
lartpc     70831
larrout        0
surf           0
infra          0
situ           0
vma            0
dtype: int64


Autres valeurs manquantes, indiqué par -1 (non renseigné)

In [301]:
print((lieux["prof"]== -1).sum())
print((lieux["surf"]== -1).sum())
print((lieux["infra"]== -1).sum())
print((lieux["situ"]== -1).sum())
print((lieux["circ"]== -1).sum())
print((lieux["plan"]== -1).sum())

211
208
923
198
5021
164


La majorité des variable ont très peu de valeurs manquantes.

On va supprimer la colonnes lartpc car elles possèdent trop de valeurs manquantes

In [303]:
lieux = lieux.drop(columns=["lartpc"])

#### VÉHICULES :

On veut garder les variables : catv, obs, obsm, choc, manv, motor, occutc, id_vehicule

In [304]:
vehicules = vehicules.drop(columns=["senc","num_veh"])

Types

In [305]:
vehicules.dtypes

Num_Acc          int64
id_vehicule     object
catv             int64
obs              int64
obsm             int64
choc             int64
manv             int64
motor            int64
occutc         float64
dtype: object

Valeurs manquantes

In [306]:
vehicules.isna().sum()

Num_Acc            0
id_vehicule        0
catv               0
obs                0
obsm               0
choc               0
manv               0
motor              0
occutc         92747
dtype: int64

Autres

In [307]:
print((vehicules["obs"] == -1).sum())
print((vehicules["obsm"] == -1).sum())
print((vehicules["choc"] == -1).sum())
print((vehicules["manv"] == -1).sum())
print((vehicules["motor"] == -1).sum())


57
46
58
37
181


On supprime la colonnes occutc qui possède trop de valeurs manquantes.

In [318]:
vehicules = vehicules.drop(columns=["occutc"])

#### USAGERS :

On souhaite garder les variables : id_usagers, id_vehicule, place, catu, grav, sexe, an_nais, trajet, secu1, secu2, secu3, locp, actp, etatp

In [309]:
usagers = usagers.drop(columns=["num_veh"])

Types

In [310]:
usagers.dtypes

Num_Acc          int64
id_usager       object
id_vehicule     object
place            int64
catu             int64
grav             int64
sexe             int64
an_nais        float64
trajet           int64
secu1            int64
secu2            int64
secu3            int64
locp             int64
actp            object
etatp            int64
dtype: object

Valeurs problématique de actp :

9 – Autre  
A – Monte/descend du véhicule  
B – Inconnue 

In [311]:
usagers["actp"] =  usagers["actp"].replace({"A": "10", "B": "9"}).apply(pd.to_numeric)

Valeurs manquantes

In [312]:
usagers.isna().sum()

Num_Acc           0
id_usager         0
id_vehicule       0
place             0
catu              0
grav              0
sexe              0
an_nais        2598
trajet            0
secu1             0
secu2             0
secu3             0
locp              0
actp              0
etatp             0
dtype: int64

In [320]:
usagers.size

1886835

In [313]:
print((usagers["trajet"] == -1).sum())
print((usagers["trajet"] == 0).sum())
print((usagers["secu1"] == -1).sum())
print((usagers["secu2"] == -1).sum())
print((usagers["secu3"] == -1).sum())
print((usagers["actp"] == -1).sum())
print((usagers["locp"] == -1).sum())
print((usagers["etatp"] == -1).sum())

2499
33303
2292
55419
123392
63253
63330
115690


On supprime les colonnes avec beaucoup de variables manquantes

In [321]:
usagers = usagers.drop(columns=["secu3","secu2","etatp"])

#### Explorations des accidents en france

- Le nombre 
- Pourcentage de gravité 
- L'heure/jour avec le plus d'accident

#### Explorations des accidents par departement 

- nb par departement
- pourcentage de gravité